# Bases de datos vectoriales

## Codificar datos a embeddings

In [1]:
!pip install -U sentence-transformers
!pip install chromadb
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.6/340.6 KB 1.1 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 3.4.1
    Uninstalling sentence-transformers-3.4.1:
      Successfully uninstalled sentence-transformers-3.4.1
  Using cached chromadb-0.6.3-py3-none-any.whl (611 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.2/65.2 KB 488.0 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 KB 849.2 kB/s eta 0:00:00a 0:00:01
  Using cached chroma_hnswlib-0.7.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.4 MB)
  Using cached tenacity-9.0.0-py3-none-any.whl (28 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 KB 733.1 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 KB 646.1 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 KB 274.2 kB/s eta 0:00:00a 0:00:01
  Using cached importlib_resource

In [6]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util

In [7]:
df = pd.read_csv('imdb_top_1000.csv')

In [9]:
df.head()

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"


In [10]:
df['text'] = df.apply(lambda x : x['Overview']+' '+x['Director']+' '+x['Star1']+' '+x['Star2']+' '+x['Star3']+' '+x['Star4'], axis=1)

In [11]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [12]:
embeddings = model.encode(df['text'],batch_size=64,show_progress_bar=True)

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

Batches: 100%|██████████| 16/16 [00:17<00:00,  1.06s/it]


In [13]:
df['embeddings'] = embeddings.tolist()

In [14]:
df['ids'] = df.index
df['ids'] = df['ids'].astype('str')

In [10]:
df

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross,text,embeddings,ids
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469",Two imprisoned men bond over a number of years...,"[-0.11102540045976639, -0.009452959522604942, ...",0
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411",An organized crime dynasty's aging patriarch t...,"[-0.10081301629543304, -0.02137056179344654, -...",1
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444",When the menace known as the Joker wreaks havo...,"[0.005191382486373186, 0.042379431426525116, -...",2
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000",The early life and career of Vito Corleone in ...,"[-0.07282990217208862, 0.007155438885092735, -...",3
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000",A jury holdout attempts to prevent a miscarria...,"[-0.08513762801885605, 0.05780969187617302, -0...",4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,https://m.media-amazon.com/images/M/MV5BNGEwMT...,Breakfast at Tiffany's,1961,A,115 min,"Comedy, Drama, Romance",7.6,A young New York socialite becomes interested ...,76.0,Blake Edwards,Audrey Hepburn,George Peppard,Patricia Neal,Buddy Ebsen,166544,NaN,A young New York socialite becomes interested ...,"[0.0284079872071743, -0.029192041605710983, -0...",995
996,https://m.media-amazon.com/images/M/MV5BODk3Yj...,Giant,1956,G,201 min,"Drama, Western",7.6,Sprawling epic covering the life of a Texas ca...,84.0,George Stevens,Elizabeth Taylor,Rock Hudson,James Dean,Carroll Baker,34075,NaN,Sprawling epic covering the life of a Texas ca...,"[0.024076644331216812, -0.036705341190099716, ...",996
997,https://m.media-amazon.com/images/M/MV5BM2U3Yz...,From Here to Eternity,1953,Passed,118 min,"Drama, Romance, War",7.6,"In Hawaii in 1941, a private is cruelly punish...",85.0,Fred Zinnemann,Burt Lancaster,Montgomery Clift,Deborah Kerr,Donna Reed,43374,"30,500,000","In Hawaii in 1941, a private is cruelly punish...","[-0.08005394786596298, 0.05756461247801781, -0...",997
998,https://m.media-amazon.com/images/M/MV5BZTBmMj...,Lifeboat,1944,NaN,97 min,"Drama, War",7.6,Several survivors of a torpedoed merchant ship...,78.0,Alfred Hitchcock,Tallulah Bankhead,John Hodiak,Walter Slezak,William Bendix,26471,NaN,Several survivors of a torpedoed merchant ship...,"[-0.02080770581960678, 0.05352265015244484, -0...",998


## Chroma

In [11]:
import chromadb
from chromadb.utils import embedding_functions

In [12]:
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key='xxxxxxxx',
    model_name = 'text-embedding-ada-002'
)

sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name = 'all-MiniLM-L6-v2')

In [14]:
chroma_client = chromadb.Client()
client_persistent = chromadb.PersistentClient(path='data_embeddings')

In [15]:
db = client_persistent.create_collection(name='movies_db', embedding_function=sentence_transformer_ef)

In [16]:
db.add(
    ids=df['ids'].tolist(),
    embeddings=df['embeddings'].tolist(),
    metadatas= df.drop(['ids','embeddings','text'],axis=1).to_dict('records')
)

In [17]:
db.peek(1)

{'ids': ['0'],
 'embeddings': array([[-1.11025400e-01, -9.45295952e-03, -6.16884008e-02,
         -1.82977691e-02, -9.83321015e-03,  6.41781390e-02,
          8.20323080e-02, -4.90679145e-02,  6.27625734e-02,
         -3.23897712e-02,  1.26143061e-02,  3.93382795e-02,
         -2.59244442e-02,  2.28466019e-02, -2.58032382e-02,
         -3.06752510e-02, -6.82935417e-02,  3.19116451e-02,
          2.50443351e-02,  7.08239302e-02,  6.20959792e-03,
         -3.05627268e-02, -1.68689210e-02, -3.70873474e-02,
          4.42673871e-03,  3.24687734e-02, -1.08997934e-02,
         -5.13101481e-02, -3.62822902e-03,  3.82995047e-02,
          2.89520007e-02,  1.47590423e-02,  1.01342574e-01,
          4.91767786e-02,  6.85054585e-02,  2.12768875e-02,
          7.06048589e-03,  2.81898957e-02,  5.03776409e-02,
         -9.52816606e-02,  1.59912705e-02,  1.71622057e-02,
          7.05167875e-02, -7.33757438e-03,  4.96426485e-02,
         -3.55564691e-02, -3.60777751e-02, -3.37920785e-02,
         -6

## Chroma embeddings

In [18]:
db_no_embeddings = client_persistent.create_collection(name='movies_db_no_embeddings', embedding_function= sentence_transformer_ef)

In [19]:
db_no_embeddings.add(
    ids=df['ids'].tolist(),
    documents=df['text'].tolist(),
    metadatas= df.drop(['ids','embeddings','text'],axis=1).to_dict('records')
)

In [20]:
db_no_embeddings.peek(1)

{'ids': ['0'],
 'embeddings': array([[-1.11025400e-01, -9.45295952e-03, -6.16884008e-02,
         -1.82977691e-02, -9.83321015e-03,  6.41781390e-02,
          8.20323080e-02, -4.90679145e-02,  6.27625734e-02,
         -3.23897712e-02,  1.26143061e-02,  3.93382795e-02,
         -2.59244442e-02,  2.28466019e-02, -2.58032382e-02,
         -3.06752510e-02, -6.82935417e-02,  3.19116451e-02,
          2.50443351e-02,  7.08239302e-02,  6.20959792e-03,
         -3.05627268e-02, -1.68689210e-02, -3.70873474e-02,
          4.42673871e-03,  3.24687734e-02, -1.08997934e-02,
         -5.13101481e-02, -3.62822902e-03,  3.82995047e-02,
          2.89520007e-02,  1.47590423e-02,  1.01342574e-01,
          4.91767786e-02,  6.85054585e-02,  2.12768875e-02,
          7.06048589e-03,  2.81898957e-02,  5.03776409e-02,
         -9.52816606e-02,  1.59912705e-02,  1.71622057e-02,
          7.05167875e-02, -7.33757438e-03,  4.96426485e-02,
         -3.55564691e-02, -3.60777751e-02, -3.37920785e-02,
         -6

In [21]:
db_no_embeddings.delete(ids=['0'])

In [22]:
db_no_embeddings.peek(1)

{'ids': ['1'],
 'embeddings': array([[-1.00813016e-01, -2.13705618e-02, -5.49226180e-02,
         -2.90707331e-02, -9.80684757e-02,  3.35139222e-02,
          3.32869366e-02,  2.21609860e-03, -2.23348439e-02,
          4.34510931e-02,  8.50618556e-02,  5.22853748e-04,
          1.83311962e-02,  3.50928912e-03, -4.52884436e-02,
         -2.04117801e-02, -3.60304117e-02,  3.35964723e-03,
         -2.14137952e-03,  1.00224065e-02, -2.88460553e-02,
         -9.99264121e-02,  1.20066423e-02,  3.99667472e-02,
         -7.68124536e-02,  5.96128628e-02, -1.17571112e-02,
          2.01192871e-02, -2.86579914e-02, -1.57587882e-02,
         -3.03114802e-02,  1.37456330e-02,  4.07512374e-02,
          4.41956967e-02,  5.25455400e-02, -5.74044585e-02,
          4.78558801e-02,  2.72374600e-02,  7.74843991e-02,
         -4.91266549e-02,  7.99732879e-02, -3.89078483e-02,
          4.35143486e-02,  2.27939505e-02,  1.53055331e-02,
         -9.79195833e-02,  3.93292634e-03,  7.53986537e-02,
          8

### Chroma Query

In [40]:
results = db_no_embeddings.query(
    query_texts=['With Brad Pitt'],
    n_results=5
)

In [43]:
results['metadatas'][0][2]

{'Certificate': 'UA',
 'Director': 'Michel Gondry',
 'Genre': 'Drama, Romance, Sci-Fi',
 'Gross': '34,400,301',
 'IMDB_Rating': 8.3,
 'Meta_score': 89.0,
 'No_of_Votes': 911664,
 'Overview': 'When their relationship turns sour, a couple undergoes a medical procedure to have each other erased from their memories.',
 'Poster_Link': 'https://m.media-amazon.com/images/M/MV5BMTY4NzcwODg3Nl5BMl5BanBnXkFtZTcwNTEwOTMyMw@@._V1_UX67_CR0,0,67,98_AL_.jpg',
 'Released_Year': '2004',
 'Runtime': '108 min',
 'Series_Title': 'Eternal Sunshine of the Spotless Mind',
 'Star1': 'Jim Carrey',
 'Star2': 'Kate Winslet',
 'Star3': 'Tom Wilkinson',
 'Star4': 'Gerry Robert Byrne'}

### Where
1. Estructura

`
{
    "metadata_field": {
        <Operator>: <Value>
    }
}
`

2. Operadores

$eq - equal to (string, int, float)

$ne - not equal to (string, int, float)

$gt - greater than (int, float)

$gte - greater than or equal to (int, float)

$lt - less than (int, float)

$lte - less than or equal to (int, float)

In [36]:
results_where = db_no_embeddings.query(
    query_texts=['a history with elves and a ring'],
    n_results=2,
    where = {
        'IMDB_Rating' : {
            "$gte":8
        }
    }
)


In [37]:
results_where

{'ids': [['10', '5']],
 'embeddings': None,
 'documents': [['A meek Hobbit from the Shire and eight companions set out on a journey to destroy the powerful One Ring and save Middle-earth from the Dark Lord Sauron. Peter Jackson Elijah Wood Ian McKellen Orlando Bloom Sean Bean',
   "Gandalf and Aragorn lead the World of Men against Sauron's army to draw his gaze from Frodo and Sam as they approach Mount Doom with the One Ring. Peter Jackson Elijah Wood Viggo Mortensen Ian McKellen Orlando Bloom"]],
 'uris': None,
 'data': None,
 'metadatas': [[{'Certificate': 'U',
    'Director': 'Peter Jackson',
    'Genre': 'Action, Adventure, Drama',
    'Gross': '315,544,750',
    'IMDB_Rating': 8.8,
    'Meta_score': 92.0,
    'No_of_Votes': 1661481,
    'Overview': 'A meek Hobbit from the Shire and eight companions set out on a journey to destroy the powerful One Ring and save Middle-earth from the Dark Lord Sauron.',
    'Poster_Link': 'https://m.media-amazon.com/images/M/MV5BN2EyZjM3NzUtNWUzMi00

In [38]:
results_where = db_no_embeddings.query(
    query_texts=['a history with elves and a ring'],
    n_results=2,
    where ={ "$and":[ {
        'IMDB_Rating' : {
            "$gte":8
        }
    },
                      {
        'Released_Year' : {
            "$ne":'2001'
        }
    }] }
)



In [39]:
results_where

{'ids': [['5', '13']],
 'embeddings': None,
 'documents': [["Gandalf and Aragorn lead the World of Men against Sauron's army to draw his gaze from Frodo and Sam as they approach Mount Doom with the One Ring. Peter Jackson Elijah Wood Viggo Mortensen Ian McKellen Orlando Bloom",
   "While Frodo and Sam edge closer to Mordor with the help of the shifty Gollum, the divided fellowship makes a stand against Sauron's new ally, Saruman, and his hordes of Isengard. Peter Jackson Elijah Wood Ian McKellen Viggo Mortensen Orlando Bloom"]],
 'uris': None,
 'data': None,
 'metadatas': [[{'Certificate': 'U',
    'Director': 'Peter Jackson',
    'Genre': 'Action, Adventure, Drama',
    'Gross': '377,845,905',
    'IMDB_Rating': 8.9,
    'Meta_score': 94.0,
    'No_of_Votes': 1642758,
    'Overview': "Gandalf and Aragorn lead the World of Men against Sauron's army to draw his gaze from Frodo and Sam as they approach Mount Doom with the One Ring.",
    'Poster_Link': 'https://m.media-amazon.com/images/

### Cargar índice de Chroma previamente creado

In [ ]:

client_persistent_2 = chromadb.PersistentClient(path="/content/data_embeddings")

In [ ]:
db_2 = client_persistent_2.get_collection('movies_db_no_embeddigs')

In [ ]:
db_2.peek(1)

# Pinecone

In [7]:
!pip install pinecone

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.9/421.9 KB 7.3 MB/s eta 0:00:00a 0:00:01


In [1]:
import os
from pinecone import Pinecone, ServerlessSpec
from dotenv import load_dotenv

/mnt/c/users/daniel.cardenas/Documents/Daniel_documents/Platzi/Curso_de_Embeddings_y_Bases_de_Datos_Vectoriales_para_NLP/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pincone_api = os.getenv('PINECONE_API_KEY')

In [3]:
load_dotenv()
pc = Pinecone(api_key=pincone_api)

In [15]:
dimension_embeddings = len(df['embeddings'][0])
dimension_embeddings

384

In [23]:
pc.create_index(
    name='movies-embeddings',
    dimension=dimension_embeddings,
    metric='cosine',
    spec=ServerlessSpec(
        cloud='aws',
        region='us-east-1'
    )
)

{
    "name": "movies-embeddings",
    "metric": "cosine",
    "host": "movies-embeddings-9gfp1sk.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [4]:
index = pc.Index('movies-embeddings')

In [26]:
df = df.fillna(' ')

In [27]:
df

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross,text,embeddings,ids
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469",Two imprisoned men bond over a number of years...,"[-0.11102540045976639, -0.009452959522604942, ...",0
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411",An organized crime dynasty's aging patriarch t...,"[-0.10081301629543304, -0.02137056179344654, -...",1
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444",When the menace known as the Joker wreaks havo...,"[0.005191382486373186, 0.042379431426525116, -...",2
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000",The early life and career of Vito Corleone in ...,"[-0.07282990217208862, 0.007155438885092735, -...",3
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000",A jury holdout attempts to prevent a miscarria...,"[-0.08513762801885605, 0.05780969187617302, -0...",4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,https://m.media-amazon.com/images/M/MV5BNGEwMT...,Breakfast at Tiffany's,1961,A,115 min,"Comedy, Drama, Romance",7.6,A young New York socialite becomes interested ...,76.0,Blake Edwards,Audrey Hepburn,George Peppard,Patricia Neal,Buddy Ebsen,166544,,A young New York socialite becomes interested ...,"[0.0284079872071743, -0.029192041605710983, -0...",995
996,https://m.media-amazon.com/images/M/MV5BODk3Yj...,Giant,1956,G,201 min,"Drama, Western",7.6,Sprawling epic covering the life of a Texas ca...,84.0,George Stevens,Elizabeth Taylor,Rock Hudson,James Dean,Carroll Baker,34075,,Sprawling epic covering the life of a Texas ca...,"[0.024076644331216812, -0.036705341190099716, ...",996
997,https://m.media-amazon.com/images/M/MV5BM2U3Yz...,From Here to Eternity,1953,Passed,118 min,"Drama, Romance, War",7.6,"In Hawaii in 1941, a private is cruelly punish...",85.0,Fred Zinnemann,Burt Lancaster,Montgomery Clift,Deborah Kerr,Donna Reed,43374,"30,500,000","In Hawaii in 1941, a private is cruelly punish...","[-0.08005394786596298, 0.05756461247801781, -0...",997
998,https://m.media-amazon.com/images/M/MV5BZTBmMj...,Lifeboat,1944,,97 min,"Drama, War",7.6,Several survivors of a torpedoed merchant ship...,78.0,Alfred Hitchcock,Tallulah Bankhead,John Hodiak,Walter Slezak,William Bendix,26471,,Several survivors of a torpedoed merchant ship...,"[-0.02080770581960678, 0.05352265015244484, -0...",998


In [28]:
from tqdm.auto import tqdm

# we will use batches of 64
batch_size=64

for i in tqdm(range(0, len(df), batch_size)):

    # find end of batch
    i_end = min(i+batch_size, len(df))
    # extract batch
    batch = df[i:i_end]
    # generate embeddings for batch
    ids = batch['ids']
    emb = batch['embeddings']
    metadata = batch.drop(['ids','embeddings','text'],axis=1).to_dict('records')

    # add all to upsert list
    to_upsert = list(zip(ids, emb, metadata))
    # upsert/insert these records to pinecone
    _ = index.upsert(to_upsert)

# check that we have all vectors in index
index.describe_index_stats()

  0%|          | 0/16 [00:00<?, ?it/s]

100%|██████████| 16/16 [00:07<00:00,  2.07it/s]


{'dimension': 384,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 1000}},
 'total_vector_count': 1000,
 'vector_type': 'dense'}

In [5]:
index.fetch(['0'])

FetchResponse(namespace='', vectors={'0': Vector(id='0', values=[-0.1110254, -0.00945295952, -0.0616884, -0.0182977691, -0.00983321, 0.064178139, 0.082032308, -0.0490679145, 0.0627625734, -0.0323897712, 0.0126143061, 0.0393382795, -0.0259244442, 0.0228466019, -0.0258032382, -0.030675251, -0.0682935417, 0.0319116451, 0.0250443351, 0.0708239302, 0.00620959792, -0.0305627268, -0.016868921, -0.0370873474, 0.00442673871, 0.0324687734, -0.0108997934, -0.0513101481, -0.00362822902, 0.0382995047, 0.028952, 0.0147590423, 0.101342574, 0.0491767786, 0.0685054585, 0.0212768875, 0.00706048589, 0.0281898957, 0.0503776409, -0.0952816606, 0.0159912705, 0.0171622057, 0.0705167875, -0.00733757438, 0.0496426485, -0.0355564691, -0.0360777751, -0.0337920785, -0.00601598248, -0.0454263873, -0.0319258235, 0.0336409807, -0.0620033108, 0.034409143, -0.0284628719, -0.0227587447, -0.00679737423, 0.0201013144, -0.0706800371, -0.0528020374, -0.0205013, 0.0246886183, 0.00475021, 0.0362827443, 0.00405066926, 0.05739

In [ ]:
index.delete(ids=['0'])

{}

In [ ]:
index.fetch(['0'])

{'namespace': '', 'vectors': {}}

### Pinecone query

In [8]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [9]:
query = 'a history of time travel'
query_vector = model.encode(query).tolist()

responses = index.query(vector=query_vector, top_k = 3, include_metadata=True)

In [10]:
responses

{'matches': [{'id': '761',
              'metadata': {'Certificate': 'U',
                           'Director': 'Mamoru Hosoda',
                           'Genre': 'Animation, Adventure, Comedy',
                           'Gross': ' ',
                           'IMDB_Rating': 7.7,
                           'Meta_score': ' ',
                           'No_of_Votes': 60368.0,
                           'Overview': 'A high-school girl named Makoto '
                                       'acquires the power to travel back in '
                                       'time, and decides to use it for her '
                                       'own personal benefits. Little does she '
                                       'know that she is affecting the lives '
                                       'of others just as much as she is her '
                                       'own.',
                           'Poster_Link': 'https://m.media-amazon.com/images/M/MV5BMzA4ZGM1NjYtMjcxY

### Filter

`
filter={
        "genre": {"$eq": "documentary"},
        "year": 2019
    }
`

The metadata filters can be combined with AND and OR:

$eq - Equal to (number, string, boolean)

$ne - Not equal to (number, string, boolean)

$gt - Greater than (number)

$gte - Greater than or equal to (number)

$lt - Less than (number)

$lte - Less than or equal to (number)

$in - In array (string or number)

$nin - Not in array (string or number)


In [11]:
query = 'a history of time travel'
query_vector = model.encode(query).tolist()

responses = index.query(vector=query_vector, top_k = 3, include_metadata=True,
                        filter={
                            "Genre":{'$ne':"Animation, Adventure, Comedy"}
                        })

In [12]:
responses

{'matches': [{'id': '47',
              'metadata': {'Certificate': 'U',
                           'Director': 'Robert Zemeckis',
                           'Genre': 'Adventure, Comedy, Sci-Fi',
                           'Gross': '210,609,762',
                           'IMDB_Rating': 8.5,
                           'Meta_score': 87.0,
                           'No_of_Votes': 1058081.0,
                           'Overview': 'Marty McFly, a 17-year-old high school '
                                       'student, is accidentally sent thirty '
                                       'years into the past in a '
                                       'time-traveling DeLorean invented by '
                                       'his close friend, the eccentric '
                                       'scientist Doc Brown.',
                           'Poster_Link': 'https://m.media-amazon.com/images/M/MV5BZmU0M2Y1OGUtZjIxNi00ZjBkLTg1MjgtOWIyNThiZWIwYjRiXkEyXkFqcGdeQXVyMTQxNzMzNDI@._V1_

### Load Index

In [ ]:
pinecone.init(api_key=pincone_api, environment="us-west4-gcp-free")

In [ ]:
index_2 = pinecone.Index('movies-emebeddings')

In [ ]:
query = 'a history of an space journey'
query_vector = model.encode(query).tolist()

responses = index_2.query(
  vector=query_vector,
  top_k=3,
  include_metadata=True,
)


In [ ]:
responses

{'matches': [{'id': '686',
              'metadata': {'Certificate': 'PG',
                           'Director': 'Philip Kaufman',
                           'Genre': 'Adventure, Biography, Drama',
                           'Gross': '21,500,000',
                           'IMDB_Rating': 7.8,
                           'Meta_score': 91.0,
                           'No_of_Votes': 56235.0,
                           'Overview': 'The story of the original Mercury 7 '
                                       'astronauts and their macho, '
                                       'seat-of-the-pants approach to the '
                                       'space program.',
                           'Poster_Link': 'https://m.media-amazon.com/images/M/MV5BOTUwMDA3MTYtZjhjMi00ODFmLTg5ZTAtYzgwN2NlODgzMmUwXkEyXkFqcGdeQXVyNjc1NTYyMjg@._V1_UX67_CR0,0,67,98_AL_.jpg',
                           'Released_Year': '1983',
                           'Runtime': '193 min',
                           'Serie